In [1]:
#%matplotlib inline
import sys
sys.version

'3.7.9 (default, Jan 24 2021, 23:57:56) \n[GCC 10.2.0]'

In [2]:
# Change the list of Test Runs in the testrun List --> Eval images will be created in root folder eval-images/

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize
import json
from matplotlib.ticker import MaxNLocator
import matplotlib

#matplotlib.use("pgf")
#matplotlib.rcParams.update({
#    "pgf.texsystem": "pdflatex",
#    'font.family': 'serif',
#    'text.usetex': True,
#    'pgf.rcfonts': False,
#})

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:
dict_for_colors = {}
get_counter_of_pid_list = []
available_colors = ['g','r','c','m','y','k']*1000000

def assign_color_to_hostname_pid(hostname, pid) -> str:
    key = hostname + pid
    if not key in dict_for_colors:
        dict_for_colors[key] = available_colors[len(dict_for_colors)]
    return dict_for_colors[key]

def get_uid_of_ri(pid, hostname, accelerator) -> int:
    key = str(pid) + str(hostname) + str(accelerator)
    try:
        val =  get_counter_of_pid_list.index(key)
        return val
    except Exception:
        get_counter_of_pid_list.append(key)
        return get_counter_of_pid_list.index(key)

def accelerator_type_from_accelerator(acc) -> str:
    if acc == "mycpu":
        return "cpu"
    if acc == "0" or acc == "1":
        return "gpu"
    if acc == "main.py": ## I don't understand this either
        return "vpu"

In [5]:
def load_data(filename): #data, trps, ps, images_folder, util

    images_folder = "../../../eval-images/" + filename.split("_")[2] +"/"

    ps = filename.split("_")
    trps = dotdict({
        'p0': int(ps[3])/1000,
        'p0t': float(ps[4]),
        'p1': int(ps[5])/1000,
        'p2': int(ps[6])/1000,
        "p2t": float(ps[7].replace(".json", ""))
    })

    with open(filename) as f:
        d = json.load(f)
    data = json_normalize(d)

    failed = [end == -1 for end in data['end']]
    data.insert(0, "failed", failed)

    if data.start.min() != 0:
        global experiment_start
        experiment_start = data.start.min()

    data.start = data.start - experiment_start
    data.end = data.end - experiment_start
    data['result.start_computation'] = data['result.start_computation'] - experiment_start
    data['result.end_computation'] = data['result.end_computation'] - experiment_start
    data['result.metadata.start'] = data['result.metadata.start'] - experiment_start
    data['result.metadata.end'] = data['result.metadata.end'] - experiment_start

    # End to End Latency
    data.insert(0,"rlat",data.end - data.start)

    # Distribution Latency - How long was it 
    data.insert(0, "dlat", data.rlat - data['result.metadata.inference_ms'])

    data.insert(0, "rfast", data.rlat <= 10000)

    data = data.loc[data['failed'] == False]

    # (semi-)unique color per runtime instance
    acc_color = [assign_color_to_hostname_pid(hostname, pid) for hostname,pid in zip(data['result.metadata.hostname'], data['result.pid'])]
    data.insert(0,"acc_color", acc_color)

    # unique id for every pid, starting with 0 (may be used as y-axis in some images)
    inst_id = [get_uid_of_ri(pid, hn, acc) for pid,hn,acc in zip(data['result.pid'],data['result.metadata.hostname'],data['result.accelerator'])]
    data.insert(0, "inst_id", inst_id)

    # String that identifies the type of accelerator
    acc_type = [accelerator_type_from_accelerator(acc) for acc in data['result.accelerator']]
    data.insert(0, "acc_type", acc_type)

    acc_name = [str(hn) + "-" + str(acc) for hn,acc in zip(data['result.metadata.hostname'], data['result.accelerator'])]
    data.insert(0, "acc_name", acc_name)
    
    # Read the gpu+queue util as well
    util = pd.read_pickle(ps[0] + "_stats.pkl")
    util['time'] = util['time'] - experiment_start
    experiment_dur = data['end'].max() * 1000
    util = util.loc[util['time'] > 0]
    util = util.loc[util['time'] < experiment_start + experiment_dur]

    util['time'] = util['time']/1000 # convert to seconds
    
    return data, trps, ps, images_folder, util

In [6]:
def generate_seconds(data, rstart=False):
    # Plot by RStart
    min, max = 0, 0
    if rstart:
        max = int(data.start.max() / 1000) # Rounds down to the latest second
        min = 0 # Per definition for the start
    else:
        # Plot by EStart
        max = int(data['result.start_computation'].max() / 1000)
        min = int(data['result.start_computation'].min() / 1000)
    data_smooth = data.rolling(1000, min_periods=1).mean()

    seconds_d = {
        # Request-Response Latency
        'rlat_min': [],
        'rlat_med': [],
        'rlat_max': [],
        # Execution Start-End Latency
        'elat_min': [],
        'elat_med': [],
        'elat_max': [],
        # Number of successful invocations
        'rsuccess': [],
        #'rfast_min': [],
        'rfast_avg': [],
        #'rfast_max': []
    }

    for i in range(min, max):
        t_min = i * 1000
        t_max = (i+1) * 1000
        # For Host Selection:
        # (data['result.metadata.hostname'] != 'sandybridge-ep') & 
        # Plot by RStart
        curr, curr_smooth = {}, {}
        if rstart:
            curr = data.loc[(data.start >= t_min) & (data.start < t_max)]
            curr_smooth = data_smooth.loc[(data.start >= t_min) & (data.start < t_max)]
        else:
        # Plot by EStart
            curr = data.loc[(data['result.start_computation'] >= t_min) & (data['result.start_computation'] < t_max)]
            curr_smooth = data_smooth.loc[(data['result.start_computation'] >= t_min) & (data['result.start_computation'] < t_max)]
        seconds_d['rlat_min'].append(curr.rlat.min())
        seconds_d['rlat_med'].append(curr.rlat.median())
        seconds_d['rlat_max'].append(curr.rlat.max())

        seconds_d['elat_min'].append(curr['result.metadata.inference_ms'].min())
        seconds_d['elat_med'].append(curr['result.metadata.inference_ms'].median())
        seconds_d['elat_max'].append(curr['result.metadata.inference_ms'].max())
        
        next10s = []
        if rstart:
            next10s = data.loc[(data.start >= t_min) & (data.start < t_min + 10000)]
        else:
        # Plot by EStart
            next10s = data.loc[(data['result.start_computation'] >= t_min) & (data['result.start_computation'] < t_min+10000)]

        #next10s = data.loc[(data.start >= t_min) & (data.start < t_min + 10000)]
        seconds_d['rsuccess'].append(next10s.shape[0]/10)
        #seconds_d['rfast_min'].append(next10s.loc[next10s['rfast']].min())
        seconds_d['rfast_avg'].append(next10s.loc[next10s['rfast']].shape[0]/10)
        #seconds_d['rfast_max'].append(next10s.loc[next10s['rfast']].max())

    seconds = pd.DataFrame(seconds_d)
    seconds_smooth = seconds.rolling(10, min_periods=1).mean()
    return seconds, seconds_smooth

In [17]:
def plot_seconds(data, xlim, latlim, reqlim, filename, opposing=False):
    fig = plt.figure(figsize=(5.4, 3.6))
    ax = plt.axes()
    ax.set_ylabel("Latency [ms]")
    ax.set_xlabel("Time since start [s]")
    ax.set_ylim(latlim[0], latlim[1])
    rlat = sns.lineplot(ax=ax, data=data, x=data.index, y='rlat_med', label="RLat")
    elat = sns.lineplot(ax=ax, data=data, x=data.index, y='elat_med', label="ELat")
    rlats = ax.fill_between(data.index, data['rlat_min'], data['rlat_max'], alpha=0.4)
    elats = ax.fill_between(data.index, data['elat_max'], data['elat_min'], alpha=0.4)
    ax.get_legend().remove()

    ax2 = ax.twinx()
    ax2.set_ylabel("Requests/s [\#]")
    ax2.set_ylim(reqlim[0], reqlim[1])
    rsuc = sns.lineplot(ax=ax2, data=data, x=data.index, y='rsuccess', label="RSuccess", color="darkgreen", alpha=0.6)
    rfast = sns.lineplot(ax=ax2, data=data, x=data.index, y='rfast_avg', label="RFast", color="crimson", alpha=0.6)
    ##ax2.fill_between(x=data.index, y1=data['rfast_min'], y2=data['rfast_max'], alpha=0.4)
    test  = sns.lineplot(ax=ax2, x=[0,trps.p0,trps.p0+trps.p1,trps.p0+trps.p1+trps.p2], y=[trps.p0t, trps.p0t, trps.p2t, trps.p2t], color="k", label="Test plan", alpha=0.6)
    
    # For the other way around
    if opposing:
        test2 = sns.lineplot(ax=ax2, x=[0,trps.p0,trps.p0+trps.p1,trps.p0+trps.p1+trps.p2], y=[trps.p2t, trps.p2t, trps.p0t, trps.p0t], color="dimgray", label="Opp. test plan", alpha=0.6)
    
    ax2.get_legend().remove()
    #ax2.yaxis.set_major_locator(MaxNLocator(integer=True))

    handles,labels = [],[]
    for ax in fig.axes:
        for h,l in zip(*ax.get_legend_handles_labels()):
            handles.append(h)
            labels.append(l)

    plt.legend(handles,labels, loc='upper left')

    #plt.legend()#bbox_to_anchor=(1, 1), borderaxespad=0) #loc="upper right")#,
    plt.tight_layout()
    plt.xlim(xlim)
    plt.savefig(images_folder + filename, dpi=1200)
    plt.close()

def plot_computation(data, color, xlim, filename):
    ax = plt.axes()
    ax.set_ylabel("Runtime ID [\#]")
    ax.set_xlabel("Time since start [s]")
    plt.hlines(y=data.inst_id, xmin=data['result.start_computation']/1000, xmax=data['result.end_computation']/1000, colors=color, linestyle="solid")
    plt.tight_layout()
    plt.xlim(xlim)
    plt.savefig(images_folder + filename, dpi=1200)
    plt.close()
    
def plot_dual_computation(onnx, onnx2, xlim, filename):
    ax = plt.axes()
    ax.set_ylabel("Runtime ID [\#]")
    ax.set_xlabel("Time since start [s]")
    plt.hlines(y=onnx.inst_id, xmin=onnx['result.start_computation']/1000, xmax=onnx['result.end_computation']/1000, colors="g", linestyle="solid", label="Rising workload")
    plt.hlines(y=onnx2.inst_id, xmin=onnx2['result.start_computation']/1000, xmax=onnx2['result.end_computation']/1000, colors="r", linestyle="solid", label="Falling workload")
    
    #ax.get_legend().remove()
    plt.legend(loc='upper left')
    
    plt.tight_layout()
    plt.xlim(xlim)
    plt.savefig(images_folder + filename, dpi=1200)
    plt.close()

def plot_gpu_queue(util, endpoint, xlim, reqlim, a1, a2, b1, gpuf, queuef, opposing=False):
    rolling = util[0:endpoint].rolling(30, min_periods=1).mean()
    
    # allGPU
    fig = plt.figure(figsize=(5.4, 3.6))
    ax = plt.axes()
    ax.set_ylabel("GPU Utilization [%]")
    ax.set_ylim(0, 101)

    if a1:
        sns.lineplot(ax=ax,data=rolling, x='time', y='util_gpu1', label="A-1")
    if a2:
        sns.lineplot(ax=ax,data=rolling, x='time', y='util_gpu2', label="A-2")
    if b1:
        sns.lineplot(ax=ax,data=rolling, x='time', y='util_gpu3', label="B-1")
    ax.get_legend().remove()

    ax2 = ax.twinx()
    ax2.set_ylabel("Requests/s [\#]")
    ax2.set_ylim(reqlim[0], reqlim[1])
    sns.lineplot(ax=ax2,x=[0,trps.p0,trps.p0+trps.p1,trps.p0+trps.p1+trps.p2], y=[trps.p0t, trps.p0t, trps.p2t, trps.p2t], color="black", label="Test plan")

    # For the other way around
    if opposing:
        test2 = sns.lineplot(ax=ax2, x=[0,trps.p0,trps.p0+trps.p1,trps.p0+trps.p1+trps.p2], y=[trps.p2t, trps.p2t, trps.p0t, trps.p0t], color="dimgray", label="Opp. test plan", alpha=0.6)
    
    
    handles,labels = [],[]
    for ax in fig.axes:
        for h,l in zip(*ax.get_legend_handles_labels()):
            handles.append(h)
            labels.append(l)

    plt.legend(handles,labels, loc='upper left')
    plt.xlim(xlim)

    plt.savefig(images_folder + gpuf, dpi=1200)
    plt.close()

    ############################################
    fig = plt.figure(figsize=(5.4, 3.6))
    ax = plt.axes()
    ax.set_ylabel("Queued Invocations [\#]")

    sns.lineplot(ax=ax,data=rolling, x='time', y='queued', label="Queued invocations")
    ax.get_legend().remove()

    ax2 = ax.twinx()
    ax2.set_ylabel("Requests/s [\#]")
    ax2.set_ylim(reqlim[0], reqlim[1])
    sns.lineplot(ax=ax2,x=[0,trps.p0,trps.p0+trps.p1,trps.p0+trps.p1+trps.p2], y=[trps.p0t, trps.p0t, trps.p2t, trps.p2t], color="black", label="Test plan")
    # For the other way around
    if opposing:
        test2 = sns.lineplot(ax=ax2, x=[0,trps.p0,trps.p0+trps.p1,trps.p0+trps.p1+trps.p2], y=[trps.p2t, trps.p2t, trps.p0t, trps.p0t], color="dimgray", label="Opp. test plan", alpha=0.6)
    
    handles,labels = [],[]
    for ax in fig.axes:
        for h,l in zip(*ax.get_legend_handles_labels()):
            handles.append(h)
            labels.append(l)

    plt.legend(handles,labels, loc='upper left')
    plt.xlim(xlim)

    plt.savefig(images_folder + queuef, dpi=1200)
    plt.close()
    


In [18]:
class TestRun():
    def __init__(self, rfast_end, xlim, latlim, reqlim, latlimp0, latlimlo, filename, double=False):
        self.rfast_end = rfast_end
        self.xlim = xlim
        self.latlim = latlim
        self.reqlim  =reqlim
        self.latlimp0 = latlimp0 
        self.latlimlo = latlimlo
        self.filename = filename
        self.data, self.trps, self.ps, self.images_folder, self.util = load_data(filename)
        self.seconds, self.seconds_smooth = generate_seconds(self.data)
        self.double = double

In [19]:
testruns = [
    TestRun(420,[0, 1050],[0, 200_000],[0,5.1],[0, 10_000],[0, 10_000],"../../../long-results/2021-08-02T15:36:25_inv_singleServerSingleGPU_120000_0.5_600000_120000_2.5.json"),
    TestRun(520,[0, 1050],[0, 200_000],[0,5.1],[0, 10_000],[0, 10_000],"../../../long-results/2021-08-02T16:09:09_inv_singleServerAllGPU_120000_1.0_600000_120000_4.0.json"),
    TestRun(620,[0, 1050],[0, 200_000],[0,5.1],[0, 10_000],[0, 10_000],"../../../long-results/2021-07-13T16:54:46_inv_singleServerAllVPUForDual_120000_1.0_600000_120000_5.0.json"),
    # rename folder!
    TestRun(620,[0, 850],[0, 70_000],[0,25],[0, 8_100],[0, 10_000],"../../../long-results/2021-07-13T16:54:46_inv_singleServerAllVPU_120000_1.0_600000_120000_5.0.json"),
    TestRun(530,[0, 850],[0, 70_000],[0,25],[0, 8_100],[0, 10_000],"../../../long-results/2021-07-03T21:00:21_inv_allIhave_120000_10_600000_120000_24.json"),
    TestRun(600,[0, 850],[0, 20_000],[0,20],[0, 8_100],[0, 10_000],"../../../long-results/2021-07-19T19:18:05_inv_allIhaveDoubleBetter_120000_5.0_600000_120000_10.0.json", True),
]

plt.ioff()

for suffix in ["png", "pgf", "pdf"]:
    print(f'Format={suffix}')

    for idx, run in enumerate(testruns):
        print(f'Round={idx}, Run={run.filename}')
        global trps
        trps = run.trps
        global images_folder
        images_folder = run.images_folder
        # Seconds parameters:
        # data, xlim, latlim, reqlim, filename, opposing=False
        # Normal Seconds
        plot_seconds(run.seconds, run.xlim, run.latlim, run.reqlim, "latency." + suffix)
        plot_seconds(run.seconds_smooth, run.xlim, run.latlim, run.reqlim, "latency_smooth." + suffix)

        # RFast
    
        plot_seconds(run.seconds_smooth.iloc[0:run.rfast_end], [0, run.rfast_end], run.latlimlo, run.reqlim, "latency_untillost_smooth." + suffix)

        # P0
        plot_seconds(run.seconds_smooth.iloc[0:120], [0, 120], run.latlimp0, run.reqlim, "latency_p0_smooth." + suffix)

        # GPU+Queue
        # enpoint is xlim, how does this look?
        # util, endpoint, xlim, reqlim, a1, a2, b1, gpuf, queuef, opposing=False)
        hasa2 = run.rfast_end > 421
        hasb1 = run.rfast_end == 530 or run.rfast_end == 600
        plot_gpu_queue(run.util, run.xlim[1], run.xlim, run.reqlim, True, hasa2, hasb1, "gpu." + suffix, "queue." + suffix)

        # Invocations    
        if run.double:
            onnx = run.data.loc[run.data['inv.runtime'] == 'onnx']
            onnx2 = run.data.loc[run.data['inv.runtime'] == 'onnx2']
            plot_dual_computation(onnx, onnx2, run.xlim, "invocations_runtimes_opp." + suffix)
            seconds_o, seconds_smooth_o = generate_seconds(onnx)
            seconds_o2, seconds_smooth_o2 = generate_seconds(onnx2)
            plot_seconds(seconds_smooth_o, run.xlim, run.latlim, run.reqlim,"latency_smooth_o1." + suffix, True)
            plot_seconds(seconds_smooth_o2, run.xlim, run.latlim, run.reqlim, "latency_smooth_o2." + suffix, True)
        else:
            plot_computation(run.data, run.data['acc_color'], run.xlim, "invocations_runtimes." + suffix)
    

Format=png
Round=0, Run=../../../long-results/2021-08-02T15:36:25_inv_singleServerSingleGPU_120000_0.5_600000_120000_2.5.json
Round=1, Run=../../../long-results/2021-08-02T16:09:09_inv_singleServerAllGPU_120000_1.0_600000_120000_4.0.json
Round=2, Run=../../../long-results/2021-07-13T16:54:46_inv_singleServerAllVPUForDual_120000_1.0_600000_120000_5.0.json
Round=3, Run=../../../long-results/2021-07-13T16:54:46_inv_singleServerAllVPU_120000_1.0_600000_120000_5.0.json
Round=4, Run=../../../long-results/2021-07-03T21:00:21_inv_allIhave_120000_10_600000_120000_24.json
Round=5, Run=../../../long-results/2021-07-19T19:18:05_inv_allIhaveDoubleBetter_120000_5.0_600000_120000_10.0.json
Format=pgf
Round=0, Run=../../../long-results/2021-08-02T15:36:25_inv_singleServerSingleGPU_120000_0.5_600000_120000_2.5.json
Round=1, Run=../../../long-results/2021-08-02T16:09:09_inv_singleServerAllGPU_120000_1.0_600000_120000_4.0.json
Round=2, Run=../../../long-results/2021-07-13T16:54:46_inv_singleServerAllVPUF